In [50]:
import numpy as np
import autograd.numpy as npa
import matplotlib as mpl
mpl.rcParams['figure.dpi']=100
import matplotlib.pylab as plt
from autograd.scipy.signal import convolve as conv
from skimage.draw import circle
import ceviche
from ceviche import fdfd_ez, jacobian, fdfd_hz
from ceviche.optimizers import adam_optimize
from ceviche.modes import insert_mode
import collections
from make_gif import make_gif
# Create a container for our slice coords to be used for sources and probes
Slice = collections.namedtuple('Slice', 'x y')

In [51]:
# The two angular frequencies
omega1=2*np.pi*200e12
omega2=2*np.pi*230e12
# Spatial resolution in meters
dl=40e-9
# Number of pixels in x-direction
Nx=240
# Number of pixels in y-direction
Ny=240
# Number of pixels in the PMLs in each direction
Npml=40
# Minimum value of the relative permittivity
epsr_train_min=-6.0
epsr_train_max=1.0
# Maximum value of the relative permittivity
epsr_design=1.0
epsr_bckgd=12.0

#------------------- Domain Constants -------------------#
# Space between the PMLs and the design region (in pixels)
space=20
# Width of the waveguide (in pixels)
wg_width=24
# Length in pixels of the source/probe slices on each side of the center point
space_slice=16
# Number of dots in a row/column
num_dots=6
# Radius of dots
circle_rad=int((Nx-2*(Npml+space))/(2*num_dots))
# Lattice constant
ra=0.6

#---------------- Optimization Constants ----------------#
# Number of epochs in the optimization 
Nsteps=500
# Step size for the Adam optimizer
step_size=5e-4

#------------------ Graphing Constants ------------------#
Emax=0
emin=-6
emax=1

In [52]:
print(omega2*2*circle_rad*dl/299792458)

3.856354840391094


In [ ]:
# set the colormap and centre the colorbar
class MidpointNormalize(mpl.colors.Normalize):
    """
    Normalise the colorbar so that diverging bars work there way either side from a prescribed midpoint value)

    e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-100, vmax=100))
    """
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        mpl.colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))
    
def real(val, outline=None, ax=None, cbar=False, cmap='RdBu', outline_alpha=0.5, vmin=None, vmax=None):
    """Plots the real part of 'val', optionally overlaying an outline of 'outline'
    """
    if ax is None:
        fig, ax = plt.subplots(1, 1, constrained_layout=True)
    
    if vmin is None:
        vmax = np.real(val).max()
        vmin = np.real(val).min()
    h = ax.imshow(np.real(val.T), cmap=cmap, origin='lower left', clim=(vmin, vmax), \
                  norm=MidpointNormalize(midpoint=0,vmin=vmin,vmax=vmax))
    
    if outline is not None:
        ax.contour(outline.T, 0, colors='k', alpha=outline_alpha)

    if cbar:
        cbar = plt.colorbar(h, ax=ax)
        cbar.set_ticks([-6, 0, 1])
        cbar.set_ticklabels(['-6', '0', '1'])
        cbar.ax.set_ylabel('Relative Permittivity')
    
    return ax

def abslt(val, outline=None, ax=None, cbar=False, cmap='magma', outline_alpha=0.5, outline_val=None, vmax=None):
    """Plots the absolute value of 'val', optionally overlaying an outline of 'outline'
    """
    if ax is None:
        fig, ax = plt.subplots(1, 1, constrained_layout=True)      
    
    if vmax is None:
        vmax = np.abs(val).max()
    h = ax.imshow(np.abs(val.T), cmap=cmap, origin='lower left', vmin=0, vmax=vmax)
    
    if outline_val is None and outline is not None: outline_val = 0.5*(outline.min()+outline.max())
    if outline is not None:
        ax.contour(outline.T, [outline_val], colors='w', alpha=outline_alpha)

    if cbar:
        cbar = plt.colorbar(h, ax=ax)
        cbar.set_ticks([])
        cbar.ax.set_ylabel('Electric Field Strength')
    
    return ax

In [ ]:
def viz_sim(epsr, source1, source2, slices=[], log=0):
    """Solve and visualize a simulation with permittivity 'epsr'
    """
    global emin, emax, Emax
    
    simulation1 = fdfd_hz(omega1, dl, epsr, [Npml, Npml])
    Ex1, _, _ = simulation1.solve(source1)
    simulation2 = fdfd_hz(omega2, dl, epsr, [Npml, Npml])
    Ex2, _, _ = simulation2.solve(source2)
    
    epsr_graph = np.array(epsr, copy=True)
    epsr_graph[Npml+space:Nx-Npml-space, Npml+space:Ny-Npml-space] = 0
    
    fig, axs = plt.subplots(1, 3, constrained_layout=True, figsize=(9,3))
    abslt(Ex1, outline=epsr_graph, ax=axs[0], cbar=False)
    abslt(Ex2, outline=epsr_graph, ax=axs[1], cbar=True)
    real(epsr, ax=axs[2], cmap='RdGy', cbar=True, vmin=emin, vmax=emax)
    for sl in slices:
        for ax in axs:
            try:
                ax.plot(sl.x*np.ones(len(sl.y)), sl.y, 'w-', alpha=0.5)
            except:
                ax.plot(sl.x, sl.y*np.ones(len(sl.x)), 'w-', alpha=0.5)
    axs[0].set_title('$\lambda_1$ = %.2f $\mu$m' % (299792458/(omega1/2/np.pi)/1e-6))
    axs[1].set_title('$\lambda_2$ = %.2f $\mu$m' % (299792458/(omega2/2/np.pi)/1e-6))
        
    if log:
        Estack = np.dstack((Ex1, Ex2))        
        if np.abs(Estack).max() > Emax:
            Emax = np.abs(Estack).max()
        np.savetxt('multiplex/tm/Ex1_%03d.csv' % log, np.abs(Ex1), delimiter=',')
        np.savetxt('multiplex/tm/Ex2_%03d.csv' % log, np.abs(Ex2), delimiter=',')
        
        if epsr.min() < emin:
            emin=epsr.min()
        if epsr.max() > emax:
            emax=epsr.max()
        np.savetxt('multiplex/tm/epsr_%03d.csv' % log, epsr, delimiter=',')
    
    return (simulation1, simulation2, axs, fig)

def mask_combine_rho(rods, bg_rho, design_region):
    """Utility function for combining the design region rho and the background rho
    """
    train = (epsr_train_max-epsr_train_min)*rods*(rods!=0).astype(np.float) + epsr_train_min*(rods!=0).astype(np.float)
    design = epsr_design*design_region*(rods==0).astype(np.float)
    bckgd = epsr_bckgd*bg_rho*(design_region==0).astype(np.float)
    return train + design + bckgd

def scale_epsrs(epsrs, rods):
    epsrs = epsrs.flatten()
    epsrs = npa.arctan(epsrs) / np.pi + 0.5 
    rho = np.zeros(rods[0].shape)
    for i in range(len(epsrs)):
        rho += epsrs[i]*rods[i]
    return rho

def epsr_parameterization(epsrs, bg_rho, design_region, rods):
    """Defines the parameterization steps for constructing rho
    """
    rods = scale_epsrs(epsrs, rods)
    return mask_combine_rho(rods, bg_rho, design_region)

In [ ]:
def init_design(epsrs, Nx=Nx, Ny=Ny, Npml=Npml, space=space, num_dots=num_dots, circle_rad=circle_rad, ra=ra):
    # Selector for each plasma rod
    rods = [np.zeros((Nx, Ny)) for i in range(num_dots**2)]
    # Selector for entire design region
    design_region = np.zeros((Nx, Ny))
    design_region[Npml+space:Nx-Npml-space, Npml+space:Ny-Npml-space] = 1
    
    # Initialize selector for circular plasma rods
    center_x = Npml+space+circle_rad
    for i in range(num_dots):
        center_y = Npml+space+circle_rad
        for j in range(num_dots):
            r, c = circle(center_x, center_y, circle_rad*ra)
            rods[j + i*num_dots][r, c] = 1
            center_y = center_y + 2*circle_rad
        center_x = center_x + 2*circle_rad
        
    return rods, design_region
    
def init_guides(Nx=Nx, Ny=Ny, Npml=Npml, space=space, wg_width=wg_width, space_slice=space_slice, wg_shift=18):
    """Initializes waveguides and sources

    space       : The space between the PML and the structure
    wg_width    : The feed and probe waveguide width
    space_slice : The added space for the probe and source slices
    """
    bg_rho = np.zeros((Nx, Ny))
        
    # Input waveguide
    bg_rho[0:Npml+space,Ny//2-wg_width//2:Ny//2+wg_width//2] = 1

    # Input probe slice
    input_slice = Slice(x=np.array(Npml+1), 
        y=np.arange(Ny//2-wg_width//2-space_slice, Ny//2+wg_width//2+space_slice))
    input_slices = [input_slice]
    
    # Output waveguide 1
    bg_rho[int(Nx-Npml-space)::,Npml+space+wg_shift:Npml+space+wg_width+wg_shift] = 1
    # Output waveguide 2
    bg_rho[int(Nx-Npml-space)::,Ny-Npml-space-wg_width-wg_shift:Ny-Npml-space-wg_shift] = 1
    
    # Output probe slice 1
    output_slice1 = (Slice(x=np.array(Nx-Npml-1), 
        y=np.arange(Npml+space-space_slice+wg_shift, Npml+space+wg_width+space_slice+wg_shift)))
    # Output probe slice 2
    output_slice2 = (Slice(x=np.array(Nx-Npml-1), 
        y=np.arange(Ny-Npml-space-wg_width-wg_shift-space_slice, Ny-Npml-space-wg_shift+space_slice)))
    output_slices = [output_slice1, output_slice2]
            
    return bg_rho, input_slices, output_slices

def init_domain(epsrs):
    rods, design_region = init_design(epsrs)
    bg_rho, input_slices, output_slices = \
        init_guides(Nx, Ny, Npml, space=space, wg_width=wg_width, space_slice=space_slice, wg_shift=18)
    return bg_rho, design_region, rods, input_slices, output_slices

In [ ]:
# Initialize the parametrization rho and the design region
epsrs = np.ones((num_dots, num_dots))*1
# epsrs = np.random.rand(num_dots, num_dots)*200-100
bg_rho, design_region, rods, input_slices, output_slices = init_domain(epsrs)

# Compute the permittivity from the design_region and the plasma rod permittivities
epsr_init = epsr_parameterization(epsrs, bg_rho, design_region, rods)

# Setup source
source1 = insert_mode(omega1, dl, input_slices[0].x, input_slices[0].y, epsr_init, m=1)
source2 = insert_mode(omega2, dl, input_slices[0].x, input_slices[0].y, epsr_init, m=1)

# Setup probe
probe1 = insert_mode(omega1, dl, output_slices[0].x, output_slices[0].y, epsr_init, m=1)
probe2 = insert_mode(omega2, dl, output_slices[1].x, output_slices[1].y, epsr_init, m=1)

# Simulate initial device
sim1, sim2, ax, fig = viz_sim(epsr_init, source1, source2, slices = input_slices + output_slices)

In [57]:
def callback_output_structure(iteration, of_list, epsrs):
    """Callback function to output fields and the structures (for making sweet gifs)"""
    epsrs = epsrs.reshape((num_dots, num_dots))
    epsr = epsr_parameterization(epsrs, bg_rho, design_region, rods)
    _, _, axs, fig = viz_sim(epsr, source1, source2, slices = input_slices + output_slices, log=iteration)
    plt.close()

In [58]:
# Define optimization objective
def mode_overlap(E1, E2):
    """Defines an overlap integral between the sim field and desired field
    """
    return npa.abs(npa.sum(npa.conj(E1)*E2))*1e6

Ex1, _, _ = sim1.solve(source1)
Ex2, _, _ = sim2.solve(source2)

E01 = mode_overlap(Ex1, probe1)
E02 = mode_overlap(Ex2, probe2)

In [59]:
def objective(epsrs):
    """Objective function called by optimizer
    
    1) Takes the density distribution as input
    2) Constructs epsr
    2) Runs the simulation
    3) Returns the overlap integral between the output wg field 
       and the desired mode field
    """
    epsrs = epsrs.reshape((num_dots, num_dots))
    epsr = epsr_parameterization(epsrs, bg_rho, design_region, rods)
    sim1.eps_r = epsr
    sim2.eps_r = epsr
    
    Ex1, _, _ = sim1.solve(source1)
    Ex2, _, _ = sim2.solve(source2)

    return mode_overlap(Ex1, probe1) / E01 * mode_overlap(Ex2, probe2) / E02

# Compute the gradient of the objective function using revere-mode differentiation
objective_jac = jacobian(objective, mode='reverse')

# Maximize the objective function using an ADAM optimizer
epsrs_optimum, _ = adam_optimize(objective, epsrs.flatten(), objective_jac,
                                 Nsteps=Nsteps, direction='max', step_size=step_size, callback=callback_output_structure)

Epoch:   1/500 | Duration: 55.14 secs | Value: 1.000000e+00
Epoch:   2/500 | Duration: 22.29 secs | Value: 4.077266e-01
Epoch:   3/500 | Duration: 42.10 secs | Value: 8.897308e-01
Epoch:   4/500 | Duration: 62.26 secs | Value: 1.469663e+00
Epoch:   5/500 | Duration: 22.80 secs | Value: 1.300392e+00
Epoch:   6/500 | Duration: 27.18 secs | Value: 1.326725e+00
Epoch:   7/500 | Duration: 23.28 secs | Value: 1.469146e+00
Epoch:   8/500 | Duration: 33.24 secs | Value: 1.749446e+00
Epoch:   9/500 | Duration: 25.59 secs | Value: 2.075764e+00
Epoch:  10/500 | Duration: 25.27 secs | Value: 1.975019e+00
Epoch:  11/500 | Duration: 29.59 secs | Value: 2.182799e+00
Epoch:  12/500 | Duration: 35.18 secs | Value: 2.134534e+00
Epoch:  13/500 | Duration: 26.76 secs | Value: 2.234643e+00
Epoch:  14/500 | Duration: 24.22 secs | Value: 2.546984e+00
Epoch:  15/500 | Duration: 23.89 secs | Value: 2.588964e+00
Epoch:  16/500 | Duration: 47.58 secs | Value: 2.679314e+00
Epoch:  17/500 | Duration: 33.07 secs | 

Epoch: 138/500 | Duration: 36.54 secs | Value: 4.868607e+00
Epoch: 139/500 | Duration: 38.89 secs | Value: 4.852755e+00
Epoch: 140/500 | Duration: 39.63 secs | Value: 4.901580e+00
Epoch: 141/500 | Duration: 37.46 secs | Value: 4.948321e+00
Epoch: 142/500 | Duration: 39.11 secs | Value: 4.963694e+00
Epoch: 143/500 | Duration: 31.21 secs | Value: 4.954433e+00
Epoch: 144/500 | Duration: 33.15 secs | Value: 4.946133e+00
Epoch: 145/500 | Duration: 32.62 secs | Value: 4.973468e+00
Epoch: 146/500 | Duration: 42.03 secs | Value: 5.008494e+00
Epoch: 147/500 | Duration: 42.73 secs | Value: 5.039499e+00
Epoch: 148/500 | Duration: 43.10 secs | Value: 5.056757e+00
Epoch: 149/500 | Duration: 42.05 secs | Value: 5.062792e+00
Epoch: 150/500 | Duration: 48.36 secs | Value: 5.060390e+00
Epoch: 151/500 | Duration: 44.55 secs | Value: 5.044497e+00
Epoch: 152/500 | Duration: 32.43 secs | Value: 5.038390e+00
Epoch: 153/500 | Duration: 44.99 secs | Value: 5.034332e+00
Epoch: 154/500 | Duration: 32.51 secs | 

Epoch: 275/500 | Duration: 30.56 secs | Value: 6.242427e+00
Epoch: 276/500 | Duration: 31.22 secs | Value: 6.239689e+00
Epoch: 277/500 | Duration: 33.05 secs | Value: 6.224036e+00
Epoch: 278/500 | Duration: 36.17 secs | Value: 6.200356e+00
Epoch: 279/500 | Duration: 29.56 secs | Value: 6.192503e+00
Epoch: 280/500 | Duration: 30.23 secs | Value: 6.186655e+00
Epoch: 281/500 | Duration: 30.55 secs | Value: 6.217435e+00
Epoch: 282/500 | Duration: 32.48 secs | Value: 6.244203e+00
Epoch: 283/500 | Duration: 35.31 secs | Value: 6.265093e+00
Epoch: 284/500 | Duration: 36.80 secs | Value: 6.269879e+00
Epoch: 285/500 | Duration: 27.19 secs | Value: 6.261219e+00
Epoch: 286/500 | Duration: 25.67 secs | Value: 6.248570e+00
Epoch: 287/500 | Duration: 25.55 secs | Value: 6.233503e+00
Epoch: 288/500 | Duration: 27.66 secs | Value: 6.237459e+00
Epoch: 289/500 | Duration: 25.52 secs | Value: 6.243583e+00
Epoch: 290/500 | Duration: 27.57 secs | Value: 6.267560e+00
Epoch: 291/500 | Duration: 30.78 secs | 

Epoch: 412/500 | Duration: 36.22 secs | Value: 6.406917e+00
Epoch: 413/500 | Duration: 31.41 secs | Value: 6.413807e+00
Epoch: 414/500 | Duration: 37.02 secs | Value: 6.420697e+00
Epoch: 415/500 | Duration: 27.27 secs | Value: 6.428227e+00
Epoch: 416/500 | Duration: 53.34 secs | Value: 6.435836e+00
Epoch: 417/500 | Duration: 37.42 secs | Value: 6.443048e+00
Epoch: 418/500 | Duration: 32.75 secs | Value: 6.450307e+00
Epoch: 419/500 | Duration: 41.31 secs | Value: 6.457613e+00
Epoch: 420/500 | Duration: 40.67 secs | Value: 6.464575e+00
Epoch: 421/500 | Duration: 42.85 secs | Value: 6.471595e+00
Epoch: 422/500 | Duration: 36.48 secs | Value: 6.479018e+00
Epoch: 423/500 | Duration: 35.17 secs | Value: 6.486334e+00
Epoch: 424/500 | Duration: 56.18 secs | Value: 6.493462e+00
Epoch: 425/500 | Duration: 24.51 secs | Value: 6.500693e+00
Epoch: 426/500 | Duration: 28.44 secs | Value: 6.507672e+00
Epoch: 427/500 | Duration: 34.65 secs | Value: 6.513880e+00
Epoch: 428/500 | Duration: 24.79 secs | 

In [70]:
# epsrs = epsr_parameterization(epsrs_optimum, bg_rho, design_region, rods)
# _, _, axs, fig = viz_sim(epsrs, source1, source2, slices = input_slices + output_slices)
obj1 = []
obj2 = []
for i in range(499, Nsteps):
    E1 = np.loadtxt('multiplex/tm/Ex1_%03d.csv' % i, delimiter=',')
    E2 = np.loadtxt('multiplex/tm/Ex2_%03d.csv' % i, delimiter=',')    
    epsr = np.loadtxt('multiplex/tm/epsr_%03d.csv' % i, delimiter=',')
    
    fig, axs = plt.subplots(1, 3, constrained_layout=True, figsize=(9,3))
    epsr_graph = np.array(epsr, copy=True)
    epsr_graph[Npml+space:Nx-Npml-space, Npml+space:Ny-Npml-space] = 0
    
#     obj1.append(mode_overlap(E1, probe1) / E01)
#     obj2.append(mode_overlap(E2, probe2) / E02)
    
#     axs[0].set_title('$\lambda_1$ = %.2f $\mu$m' % (299792458/(omega1/2/np.pi)/1e-6))
#     axs[1].set_title('$\lambda_2$ = %.2f $\mu$m' % (299792458/(omega2/2/np.pi)/1e-6))
    for ax in axs:
        ax.set_xlabel('')
        ax.set_ylabel('')
        ax.set_yticks([])
        ax.set_xticks([])
#     for ax in axs[1][0:2]:
#         ax.set_xlabel('Epoch')
#         ax.set_ylabel('Percent Change in Objective')
    
    abslt(E1, outline=epsr_graph, ax=axs[0], cbar=False, vmax=1e-7)
#     axs[1,0].plot(obj1)
    abslt(E2, outline=epsr_graph, ax=axs[1], cbar=True, vmax=1e-7)
#     axs[1,1].plot(obj2)
    real(epsr, ax=axs[2], cmap='RdGy', cbar=True, vmin=-6, vmax=1)
#     axs[-1,-1].axis('off')
    plt.savefig('multiplex/tm/normalized_%03d.png' % i, dpi=70)
    plt.close()

In [ ]:
grid = (epsr_train_max-epsr_train_min) * (npa.arctan(epsrs_optimum)/np.pi + 0.5) + epsr_train_min
print(grid)
make_gif('multiplex_te.gif', 475, 'multiplex/te')